# Train ReLayNet

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import h5py

from relaynet_pytorch.relay_net import ReLayNet
from relaynet_pytorch.data_utils import get_imdb_data
from helper import * # import helper functions

#torch.set_default_tensor_type('torch.FloatTensor')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
Label = h5py.File('datasets/OCTData/label.h5', 'r')
a_group_key = list(Label.keys())[0]
Label = list(Label[a_group_key])
Label = np.squeeze(np.asarray(Label))
print(Label.shape)
weights = Label[:, 1, 0:512, :]
Label = Label[:, 0, 0:512, :]
print(weights.shape)
print(Label.shape)
# print(np.unique(weights))
print(np.unique(Label))

(171, 2, 512, 64)
(171, 512, 64)
(171, 512, 64)
[ 0.  1.  2.  3.  4.  5.  6.  7.]


In [3]:
train_data, test_data = get_imdb_data()
print("Train size: %i" % len(train_data))
print("Test size: %i" % len(test_data))
# print(train_data[0])

(171, 512, 64)
(171, 512, 64)
Train size: 136
Test size: 35


In [4]:
from relaynet_pytorch.relay_net import ReLayNet
from relaynet_pytorch.solver import Solver
bs = 1
train_loader = torch.utils.data.DataLoader(train_data, batch_size=bs, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(test_data, batch_size=bs, shuffle=False, num_workers=4)

param ={
        'num_channels':1,
        'num_filters':64,
        'kernel_h':3,
        'kernel_w':7,
        'kernel_c': 1,
        'stride_conv':1,
        'pool':2,
        'stride_pool':2,
        'num_class':8
    }
# for batch in train_loader:
#     print(batch[0].size())

exp_dir_name = 'RawOCT'
CUDA_LAUNCH_BLOCKING=1
relaynet_model = ReLayNet(param)
solver = Solver(optim_args={"lr": 1e-2})
solver.train(relaynet_model, train_loader, val_loader, log_nth=1, num_epochs=2, exp_dir_name=exp_dir_name)

START TRAIN.
torch.Size([1, 1, 512, 64]) torch.Size([1, 512, 64]) torch.Size([1, 8, 512, 64])


RuntimeError: cuda runtime error (30) : unknown error at /home/sim/install/pytorch/aten/src/THC/THCGeneral.cpp:70

In [ ]:
# encoded_target = torch.zeros(4, 7, 512, 64)
# target = torch.ones(4, 512, 64, dtype=torch.long)
# print(encoded_target.shape)
# print(target.size())
# print(target.unsqueeze(1).size())
# encoded_target.scatter_(1, target.unsqueeze(1), 1)

Help from: https://github.com/pytorch/pytorch/issues/2646

In [ ]:
# torch.zeros(3, 4).scatter_(0, torch.LongTensor([[2, 2, 2],[1, 1,1],[0, 0,0]]), torch.arange(1,13).resize_(3, 4))

In [ ]:
# print(torch.arange(1,13).resize_(3, 4))
# torch.zeros(3, 4).scatter_(1, torch.LongTensor([[2, 2, 2, 2],[1, 1,1,1],[0, 0,0,0]]), torch.arange(1,13).resize_(3, 4))

In [ ]:
# torch.zeros(1, 2).scatter_(1, torch.LongTensor([[0, 0],[]), torch.arange(1,3).resize_(1, 2))